In [ ]:
#for indoor

import numpy as np
import cv2
import os
import torch
import torch.utils.data as data
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from skimage.metrics import structural_similarity as ssim

# Data Loading and Processing
class DehazingDataset(data.Dataset):
    def init(self, hazy_images_path, clean_images_path, transform=None):
        self.hazy_images = sorted([os.path.join(hazy_images_path, f)
                                   for f in os.listdir(hazy_images_path)
                                   if f.endswith(('.jpg', '.jpeg', '.png'))])
        self.clean_images = sorted([os.path.join(clean_images_path, f)
                                    for f in os.listdir(clean_images_path)
                                    if f.endswith(('.jpg', '.jpeg', '.png'))])
        self.size = min(len(self.hazy_images), len(self.clean_images))
        self.hazy_images = self.hazy_images[:self.size]
        self.clean_images = self.clean_images[:self.size]
        self.filter_files()
        self.size = len(self.hazy_images)
        self.transform = transform

    def filter_files(self):
        hazy_ims = []
        clean_ims = []
        for hazy_img_path, clean_img_path in zip(self.hazy_images, self.clean_images):
            hazy = Image.open(hazy_img_path)
            clean = Image.open(clean_img_path)
            if hazy.size == clean.size:
                hazy_ims.append(hazy_img_path)
                clean_ims.append(clean_img_path)
        self.hazy_images = hazy_ims
        self.clean_images = clean_ims

    def getitem(self, index):
        hazy_img = self.rgb_loader(self.hazy_images[index])
        clean_img = self.rgb_loader(self.clean_images[index])
        if self.transform:
            hazy_img = self.transform(hazy_img)
            clean_img = self.transform(clean_img)
        return hazy_img, clean_img

    def rgb_loader(self, path):
        with open(path, 'rb') as f:
            img = Image.open(f)
            return img.convert('RGB')

    def len(self):
        return self.size

    def show_grayscale_image(self, tensor_image):
        np_image = tensor_image.squeeze(0).numpy()
        np_image = np.clip(np_image, 0, 1)
        plt.imshow(np_image, cmap='gray')
        plt.axis('off')
        plt.show()

# Dehazing Model
def adjust_brightness_contrast(image, brightness=0, contrast=21):
    """Adjust the brightness and contrast of an image."""
    brightness = int((brightness - 0) * 255 / 100)  # Convert brightness to scale
    contrast = int((contrast - 0) * 127 / 100)  # Convert contrast to scale
    return cv2.convertScaleAbs(image, alpha=1 + contrast / 127.0, beta=brightness)

def resize_image(image, size):
    return cv2.resize(image, size, interpolation=cv2.INTER_LINEAR)

def extract_features(image):
    hsv_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    brightness = hsv_image[:, :, 2]
    saturation = hsv_image[:, :, 1]

    mean_brightness = np.mean(brightness)
    mean_saturation = np.mean(saturation)

    brightness_sum = mean_brightness + mean_saturation
    if brightness_sum == 0:
        brightness_sum = 1e-11

    eta = mean_brightness / brightness_sum
    chi = mean_saturation / brightness_sum

    return eta, chi

def calculate_beta(eta, chi, c1, c2, c3):
    epsilon = np.random.normal(0, 0.01)  # Small noise
    return c1 + c2 * eta + c3 * chi + epsilon

def refined_transmission_map(depth_map, beta):
    tr_map = np.exp(-beta * depth_map)
    return tr_map

def improved_dehaze_image(image, depth_map, c1, c2, c3, gamma=1.0, brightness=2, contrast=5):
    eta, chi = extract_features(image)
    beta = calculate_beta(eta, chi, c1, c2, c3)
    tr_map = refined_transmission_map(depth_map, beta)

    restored_image = image / (tr_map + 1e-5)
    restored_image = np.clip(restored_image, 0, 1)
    # Adjust brightness and contrast
    restored_image_uint8 = (restored_image * 255).astype(np.uint8)
    restored_image_adjusted = adjust_brightness_contrast(restored_image_uint8, brightness, contrast)
    gamma_corrected_image = np.power(restored_image_adjusted / 255.0, 1 / gamma)

    return gamma_corrected_image

def load_dataset(clear_folder, hazy_folder, size=(256, 256), max_images=200):
    clear_images = []
    hazy_images = []

    clear_files = sorted(os.listdir(clear_folder))[:max_images]
    hazy_files = sorted(os.listdir(hazy_folder))[:max_images]

    if not clear_files or not hazy_files:
        raise ValueError("One or both folders are empty or contain no valid image files.")

    for img_name in clear_files:
        img_path = os.path.join(clear_folder, img_name)
        clear_img = cv2.imread(img_path)
        if clear_img is None:
            continue
        clear_img = cv2.cvtColor(clear_img, cv2.COLOR_BGR2RGB).astype(np.float32) / 255.0
        clear_img = resize_image(clear_img, size)
        clear_images.append(clear_img)

    for img_name in hazy_files:
        img_path = os.path.join(hazy_folder, img_name)
        hazy_img = cv2.imread(img_path)
        if hazy_img is None:
            continue
        hazy_img = cv2.cvtColor(hazy_img, cv2.COLOR_BGR2RGB).astype(np.float32) / 255.0
        hazy_img = resize_image(hazy_img, size)
        hazy_images.append(hazy_img)

    min_length = min(len(clear_images), len(hazy_images))
    clear_images = clear_images[:min_length]
    hazy_images = hazy_images[:min_length]

    return np.array(clear_images), np.array(hazy_images)

# Function to display images side by side
def display_images_side_by_side(images, titles=None):
    """Display a list of images side by side."""
    fig, axes = plt.subplots(1, len(images), figsize=(15, 5))
    for i, image in enumerate(images):
        axes[i].imshow(image)
        axes[i].axis('off')
        if titles:
            axes[i].set_title(titles[i])
    plt.show()

def main(clear_folder, hazy_folder, output_folder):
    clear_images, hazy_images = load_dataset(clear_folder, hazy_folder)

    if len(clear_images) == 0 or len(hazy_images) == 0:
        raise ValueError("Image arrays are empty. Please check your image paths.")

    train_clear, test_clear, train_hazy, test_hazy = train_test_split(
        clear_images, hazy_images, test_size=0.1, random_state=42)

    train_features = []
    train_beta_values = []
   
    # Training the regression model
    for clear_img, hazy_img in zip(train_clear, train_hazy):
        eta, chi = extract_features(clear_img)
        depth_map = np.abs(clear_img - hazy_img)
        ratio = hazy_img / (clear_img + 1e-5)
        beta = -np.log(np.mean(ratio)) / np.mean(depth_map)
        train_features.append([eta, chi])
        train_beta_values.append(beta)

    train_features = np.array(train_features)
    train_beta_values = np.array(train_beta_values)
    train_features = train_features.reshape(-1, 2)

    regressor = LinearRegression()
    regressor.fit(train_features, train_beta_values)

    c1, c2, c3 = regressor.intercept_, regressor.coef_[0], regressor.coef_[1]

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    total_ssim = 0
    num_images = len(test_clear)

    for i, (test_clear_img, test_hazy_img) in enumerate(zip(test_clear, test_hazy)):
        depth_map = np.abs(test_clear_img - test_hazy_img)
        restored_img = improved_dehaze_image(test_hazy_img, depth_map, c1, c2, c3, gamma=1.0, brightness=10, contrast=1)

        restored_img_uint8 = (restored_img * 255).astype(np.uint8)
        cv2.imwrite(os.path.join(output_folder, f'restored_image_{i}.png'), cv2.cvtColor(restored_img_uint8, cv2.COLOR_RGB2BGR))

        # Ensure that the SSIM window size is compatible with your image size
        ssim_value = ssim(test_clear_img, restored_img, win_size=25, channel_axis=-1, data_range=41.0)
        total_ssim += ssim_value
        display_images_side_by_side([test_hazy_img, restored_img, test_clear_img], titles=["Hazy Image", "Dehazed Image", "Clear Image"])

    print("Mean SSIM value:", total_ssim / num_images)

# Integration
if __name__ == "__main__":
    clear_folder = '/kaggle/input/i-haze/I-HAZE/val/clear'
    hazy_folder = '/kaggle/input/i-haze/I-HAZE/val/hazy'
    output_folder = '/kaggle/working/'

    # Run the dehazing model
    main(clear_folder, hazy_folder, output_folder)